# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

Import the dataset with: 

$ mongoimport --type json -d uk_food -c establishments --drop --jsonArray Resources/establishments.json



2023-03-19T14:42:41.389-0500    connected to: mongodb://localhost/
2023-03-19T14:42:41.392-0500    dropping: uk_food.establishments
2023-03-19T14:42:42.506-0500    39779 document(s) imported successfully. 0 document(s) failed to import.


In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
import pandas as pd

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'config', 'fruits_db', 'local', 'met', 'uk_food']


In [4]:
# assign the uk_food database to a variable name
food_db = mongo['uk_food']

In [5]:
# review the collections in our new database
print(food_db.list_collection_names())

['establishments']


In [6]:
# review a document in the establishments collection
food_db.establishments.find_one()

{'_id': ObjectId('6417b66da80587c0f3cd9c90'),
 'FHRSID': 1034540,
 'ChangesByServerID': 0,
 'LocalAuthorityBusinessID': 'PI/000078691',
 'BusinessName': 'The Coastguard Inn',
 'BusinessType': 'Pub/bar/nightclub',
 'BusinessTypeID': 7843,
 'AddressLine1': 'The Bay',
 'AddressLine2': 'St Margarets Bay',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'PostCode': 'CT15 6DY',
 'Phone': '',
 'RatingValue': '5',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingDate': '2022-08-17T00:00:00',
 'LocalAuthorityCode': '182',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'scores': {'Hygiene': 0, 'Structural': 5, 'ConfidenceInManagement': 0},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': '1.387974', 'latitude': '51.152225'},
 'RightToReply': '',
 'Distance': 4587.347174863443,
 'NewRatingPending': False,
 'meta': {'dataSource': None,
  'extractDate': '0001-01-01T00:00:00',
  'itemCount': 0,
  'returncode'

In [7]:
# assign the collection to a variable
establishment_collection = food_db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [8]:
# Create a dictionary for the new restaurant data
Penang_restaurant ={
        "BusinessName":"Penang Flavours",
        "BusinessType":"Restaurant/Cafe/Canteen",
        "BusinessTypeID":"",
        "AddressLine1":"Penang Flavours",
        "AddressLine2":"146A Plumstead Rd",
        "AddressLine3":"London",
        "AddressLine4":"",
        "PostCode":"SE18 7DY",
        "Phone":"",
        "LocalAuthorityCode":"511",
        "LocalAuthorityName":"Greenwich",
        "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
        "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
        "scores":{
            "Hygiene":"",
            "Structural":"",
            "ConfidenceInManagement":""
        },
        "SchemeType":"FHRS",
        "geocode":{
            "longitude":"0.08384000",
            "latitude":"51.49014200"
        },
        "RightToReply":"",
        "Distance":4623.9723280747176,
        "NewRatingPending":True
    }

In [9]:
#Delete existing

establishment_collection.delete_one(Penang_restaurant)

In [10]:
# Insert the new restaurant into the collection
establishment_collection.insert_one(Penang_restaurant)

In [11]:
# Check that the new restaurant was inserted
query = {"BusinessName":"Penang Flavours"}
results = establishment_collection.find(query)

for row in results:
    print(row)

{'_id': ObjectId('6417b6b1930fe7a4c513dd3b'), 'BusinessName': 'Penang Flavours', 'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': '', 'AddressLine1': 'Penang Flavours', 'AddressLine2': '146A Plumstead Rd', 'AddressLine3': 'London', 'AddressLine4': '', 'PostCode': 'SE18 7DY', 'Phone': '', 'LocalAuthorityCode': '511', 'LocalAuthorityName': 'Greenwich', 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk', 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk', 'scores': {'Hygiene': '', 'Structural': '', 'ConfidenceInManagement': ''}, 'SchemeType': 'FHRS', 'geocode': {'longitude': '0.08384000', 'latitude': '51.49014200'}, 'RightToReply': '', 'Distance': 4623.972328074718, 'NewRatingPending': True}


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [12]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {"BusinessType": "Restaurant/Cafe/Canteen"}
fields = {"BusinessTypeID", "BusinessType"}
sorts = [("BusinessTypeID", 1)]
limit = 5

query_results = list(establishment_collection.find(query,fields).sort(sorts).limit(limit))
pd.DataFrame(query_results)

,_id,BusinessType,BusinessTypeID
0,6417b66da80587c0f3cd9c9b,Restaurant/Cafe/Canteen,1
1,6417b66da80587c0f3cd9ca0,Restaurant/Cafe/Canteen,1
2,6417b66da80587c0f3cd9c98,Restaurant/Cafe/Canteen,1
3,6417b66da80587c0f3cd9c95,Restaurant/Cafe/Canteen,1
4,6417b66da80587c0f3cd9c99,Restaurant/Cafe/Canteen,1


3. Update the new restaurant with the `BusinessTypeID` you found.

In [13]:
# Update the new restaurant with the correct BusinessTypeID
update_value = {'$set': {"BusinessTypeID": 1}}
establishment_collection.update_one(query, update_value)

In [14]:
# Confirm that the new restaurant was updated
query = {"BusinessName":"Penang Flavours"}
results = establishment_collection.find(query)

for row in results:
    print(row)

{'_id': ObjectId('6417b6b1930fe7a4c513dd3b'), 'BusinessName': 'Penang Flavours', 'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': '', 'AddressLine1': 'Penang Flavours', 'AddressLine2': '146A Plumstead Rd', 'AddressLine3': 'London', 'AddressLine4': '', 'PostCode': 'SE18 7DY', 'Phone': '', 'LocalAuthorityCode': '511', 'LocalAuthorityName': 'Greenwich', 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk', 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk', 'scores': {'Hygiene': '', 'Structural': '', 'ConfidenceInManagement': ''}, 'SchemeType': 'FHRS', 'geocode': {'longitude': '0.08384000', 'latitude': '51.49014200'}, 'RightToReply': '', 'Distance': 4623.972328074718, 'NewRatingPending': True}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [15]:
# Find how many documents have LocalAuthorityName as "Dover"
query = {'LocalAuthorityName': {'$regex': 'Dover'}}
dover_collection = list(establishment_collection.find(query))
print(len(dover_collection))

994


In [16]:
# Delete all documents where LocalAuthorityName is "Dover"
query = {'LocalAuthorityName': {'$regex': 'Dover'}}
establishment_collection.delete_many(query)

In [17]:
# Check if any remaining documents include Dover
establishment_collection.count_documents(query)

0

In [18]:
# Check that other documents remain with 'find_one'
establishment_collection.find_one()

{'_id': ObjectId('6417b66da80587c0f3cd9f6d'),
 'FHRSID': 1043695,
 'ChangesByServerID': 0,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'PostCode': 'CT19 6BL',
 'Phone': '',
 'RatingValue': '5',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingDate': '2018-04-04T00:00:00',
 'LocalAuthorityCode': '188',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'scores': {'Hygiene': 5, 'Structural': 5, 'ConfidenceInManagement': 5},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': '1.195625', 'latitude': '51.083812'},
 'RightToReply': '',
 'Distance': 4591.765489457773,
 'NewRatingPending': False,
 'meta': {'dataSource': None,
  'extractDate': '0001-01-01

5. Some of the number values are stored as strings, when they should be stored as numbers. Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [19]:
# Change the data type from String to Decimal for longitude
establishment_collection.update_many({}, [{'$set':{ "geocode.longitude" : {'$toDouble': "$geocode.longitude"}}}])

In [20]:
# Change the data type from String to Decimal for latitude
establishment_collection.update_many({}, [{'$set':{ "geocode.latitude" : {'$toDouble': "$geocode.latitude"}}}])

In [21]:
# Check that the coordinates are now numbers
type(establishment_collection.find_one()['geocode']['longitude'])

float